In [139]:
# ссылка на репозиторий: https://github.com/AnastasiaKoshkina/ML
# доступ публичный, так как непонятно, каким пользователям гитхаба и/или на какие почты давать доступ

#подгружаем pandas
import pandas as pd 
#подгружаем numpy - понадобится для приведения дат к месяцам
import numpy as np 

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [140]:
#ЗАДАЧА 1

#загружаем первый файл с информацией о посетителях
peopleData = pd.read_csv("CASC_Constant.csv", sep = ",")
peopleData = peopleData.drop(columns=["ActivationDate","SubscribedEmail","SubscribedPush"]) #удаление лишних столбцов
print(peopleData.head())

#загружаем второй файл с информацией о транзакциях
transactions = pd.read_csv("casc-resto-small.csv", sep = ",")
print(transactions.head())
print(transactions.describe())

   CustomerId   Age     Sex
0     2728183  21.0  Female
1     2728198  21.0  Female
2     2728306  21.0  Female
3     2728178  22.0    Male
4     2728322  22.0    Male
   CustomerID  Restaurant      RKDate                    RegionName  \
0     2898197         391  2017-07-16  Москва и Московская область    
1     2903215          43  2015-04-07  Москва и Московская область    
2     2748887          43  2015-05-22  Москва и Московская область    
3     2862077          46  2015-03-05  Москва и Московская область    
4     2862077          46  2015-03-11  Москва и Московская область    

   BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  
0  TGI FRIDAYS      NON ALCOHOL         2          2                   2.0  
1     IL Патио      NON ALCOHOL         1          5                   5.0  
2     IL Патио      NON ALCOHOL         1          5                   5.0  
3     IL Патио      NON ALCOHOL         1          5                   5.0  
4     IL Патио      

In [141]:
#ЗАДАЧА 2

transactions.RKDate = pd.to_datetime(transactions.RKDate) # преобразование из строки в дату - т.к. дата в датасете была строкового типа
print(type(transactions.RKDate[0]))
transactions = transactions.sort_values("RKDate") # сортировка по возрастанию
print(transactions.tail())


# Функция для расчета, был ли визит
def VisitHap(obj):
    result = 0
    for index, item in obj.iterrows():
        if pd.to_datetime("2017-12-31") >= item['RKDate'] >= pd.to_datetime("2017-07-01"):
            result = 1 #если визит был в указанном диапазоне - возвращаем True. Иначе - False            
    return result

# Функция для расчета частоты прихода клиента   
def FunctionFrequency(obj):
    # Считаем 2017 07 01  - дата первого визита 
    # Делим количество визитов на разницу в месяцах
    # return количество
    correct_date = obj[obj['RKDate'] < pd.to_datetime("2017-07-01")]
    if not correct_date.empty:
        months = (pd.to_datetime("2017-07-01") - correct_date["RKDate"].min()) / np.timedelta64(1,'M')
        visits =  len(correct_date.index) 
        result = visits / months
    else:
        result = 0
    return result 


# Функция для расчета среднего чека 
def FunctionMonetaryValue(obj):
    # средний чек клиента по всем покупкам до 2017-07-01
    # для каждой запсии: если дата меньше 2017 - 07 - 01 то суммируем чек, считаем чеки, потом считаем среднее 
    count = 0 
    money = 0
    for index, item in obj.iterrows():
        if item['RKDate'] < pd.to_datetime("2017-07-01"):
            count += 1
            money += item['SummBasic']
    if count == 0:
        result = 0
        return result
    result = money / count  
    count = 0 
    money = 0
    return result   

# Функция для расчета новизны - разницы между последним визитом клиента и фиксированной датой
def FunctionRecency(obj):
    #  
    correct_date = obj[obj['RKDate'] < pd.to_datetime("2017-07-01")]
    if not correct_date.empty:
        result = (pd.to_datetime("2017-07-01") - correct_date["RKDate"].max()) / np.timedelta64(1,'D')
    else:
        result = 0 
    return result



transactionsID = transactions.groupby('CustomerID') # groupby по ID клиента

for index, item in transactions.iterrows():  # расчет RFM переменных и расчет переменной Y 
    transactions.loc[index,'Recency'] = FunctionRecency(transactionsID.get_group(item['CustomerID']))
    transactions.loc[index,'Frequency'] = FunctionFrequency(transactionsID.get_group(item['CustomerID']))
    transactions.loc[index,'MonetValue'] = FunctionMonetaryValue(transactionsID.get_group(item['CustomerID']))
    transactions.loc[index,'Visithappen'] = VisitHap(transactionsID.get_group(item['CustomerID'])) # добавление переменной Y
    #print(type(FunctionVisitHappen(transactionsID.get_group(item['CustomerID']))))  # добавление переменной Y
print(transactions.info())
    
transactions = transactions.drop_duplicates(subset=['CustomerID'])  # избавление от повторов


transactions = transactions.drop(columns=["RKDate", "Restaurant", "RegionName","BrandsNames", "DishCategoryName", "Quantity", "SummBasic", "SummAfterPointsUsage"])  # удаление лишних столбцов

print(transactions.info())

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
     CustomerID  Restaurant     RKDate                    RegionName  \
209     2767598         450 2017-11-22  Москва и Московская область    
773     2768840         183 2017-11-29                   Новосибирск   
717     2821271         129 2017-12-28  Москва и Московская область    
803     2750656         295 2017-12-29  Москва и Московская область    
849     2748926         390 2017-12-30  Москва и Московская область    

    BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  
209   Мама Раша             FOOD         1          9                  7.00  
773    IL Патио      NON ALCOHOL         1         10                 10.00  
717    IL Патио             FOOD         1         10                 10.00  
803      Шикари             FOOD         1         10                  6.87  
849    IL Патио             FOOD         1         10                 10.00  
<class 'pandas.core.frame.DataFrame'>
Int64Index

In [142]:
# ЗАДАНИЕ 3 
#добавление данных из второй таблицы

peopleData.rename(columns={'CustomerId': 'CustomerID'}, inplace=True)    # переименовываем столбец – изначально они не совпадают 
MainData = transactions.merge(peopleData) # соединяем обе таблицы 
MainData = MainData.dropna() # убираем пустые значения -  не все клиенты указали гендер
MainData = MainData.replace({'Sex': {'Male': True,'Female': False}}) # Меняем гендеры на тип bool

print(MainData.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 461
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CustomerID   418 non-null    int64  
 1   Recency      418 non-null    float64
 2   Frequency    418 non-null    float64
 3   MonetValue   418 non-null    float64
 4   Visithappen  418 non-null    float64
 5   Age          418 non-null    float64
 6   Sex          418 non-null    bool   
dtypes: bool(1), float64(5), int64(1)
memory usage: 23.3 KB
None


In [143]:
# ЗАДАНИЕ 4 
#деление на тестовую и обучающую выборки
trg = MainData['Visithappen']  # Отделяем от выборки значение зависимой переменной
trn = MainData.drop(columns=['Visithappen'], axis=1)
Xtrn, Xtest, Ytrn, Ytest = train_test_split(trn, trg, test_size = 0.2) # Разбиваем выборку на тестовую и обучающую

print(Xtrn.head())
Ytrn

     CustomerID  Recency  Frequency  MonetValue   Age    Sex
176     2746541    801.0   0.037999        10.0  25.0  False
329     2888912    618.0   0.049251        10.0  37.0   True
9       2741098    834.0   0.236995        10.0  28.0  False
380     2823761    480.0   0.063410        10.0  30.0   True
107     2885186    733.0   0.108445        10.0  28.0  False


176    0.0
329    0.0
9      0.0
380    0.0
107    0.0
      ... 
10     0.0
40     0.0
135    0.0
170    0.0
337    0.0
Name: Visithappen, Length: 334, dtype: float64

In [144]:
# ЗАДАНИЕ 5
# обучение модели

logistic = LogisticRegression()
logistic.fit(Xtrn, Ytrn) # обучение модели
logistic.score(Xtrn, Ytrn) # оценка модели
#print('Coefficient: \n', logistic.coef_)
#print('Intercept: \n', logistic.intercept_)
#print('R² Value: \n', logistic.score(Xtrn, Ytrn))

0.9940119760479041

In [145]:
# ЗАДАНИЕ 6 
#предсказывание Y

prediction = pd.Series(logistic.predict(Xtest)) # Предсказывание значения у и конвертация
prediction = prediction.rename('Visithappen') # переименовение, выборки хранились в одинаковом виде

In [146]:
# ЗАДАНИЕ 7
# подсчет метрики

report = classification_report(Ytest, prediction) # Считаем метрики модели
report

'              precision    recall  f1-score   support\n\n         0.0       0.99      0.99      0.99        80\n         1.0       0.75      0.75      0.75         4\n\n    accuracy                           0.98        84\n   macro avg       0.87      0.87      0.87        84\nweighted avg       0.98      0.98      0.98        84\n'